In [5]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
import pickle
from sklearn.ensemble import GradientBoostingClassifier
import json

def rlmse_score(y_test, y_hat):
    return np.sqrt(mean_squared_log_error(y_test, y_hat))

def rlmse_scorer(estimator, X, y):
    y_hat = estimator.predict(X)
    
    return rlmse_score(y, y_hat)

In [6]:
df_test = pd.read_csv('test_3_col.csv', sep='\t', index_col='id')

In [7]:
df_test.head(5)

,Unnamed: 0,name,description
id,,,
285ea2e9935ccdeb8378c6a5,4,Стремянка,"Стремянка трехсекционная,3×4"
adfb73820bbb831257df6e95,9,Плита электрическая,Плита ЭВИ 5120. Работают 3 конфорки и духовка....
783025601c36202f633fc6a5,15,Диски R17 Rays MazdaSpeed Touring 5х114.3 7J,Все лоты привезены с площадок и аукционов Япон...
2f0cd2d2e15dc90afd847f95,19,Batman Archam Knight,NaN
5c23a37902855a20172845a5,20,Ледобур для зимней рыбалки,Отдаю ледобур времён с.с.с.р диаметр 130 мм бе...


In [8]:
df_train = pd.read_csv('train_4_col.csv', sep='\t', index_col='id')

In [9]:
df_train.head(5)

,Unnamed: 0,name,description,price
id,,,,
3edeb34cf93f490ff760af85,0,Сумка DG,NaN,199900
c98febd50dad3cc0ffc86085,1,Комплект,8-12 лет,35000
ade01e13912a46a99134cc75,2,Пальтишко демисезонное,"На девочку 1,5 г,состояние хорошее",30000
ab3e6941c11304c1519aef75,3,Attipas,"Размер-135mm, euro-22.5.Прочная, мягкая, не ск...",80000
252452a91c944a22c276d995,5,Жилет теплый,NaN,50000


In [10]:
def preprocess_data(df_input):
    df_output = df_input.copy()
    df_output = df_output.drop('description', axis = 1)
    df_output = df_output.loc[:, ~df_output.columns.str.contains('^Unnamed')]
    df_output['name'] = df_output.name.fillna('')
    df_output['name'] = df_output['name'].str.lower()
    df_output['name'] = df_output['name'].str.replace(u'[^а-яa-z -]','')
    
    return df_output

In [11]:
df_train_proc = preprocess_data(df_train)

In [12]:
df_train_proc.head()

,name,price
id,,
3edeb34cf93f490ff760af85,сумка dg,199900
c98febd50dad3cc0ffc86085,комплект,35000
ade01e13912a46a99134cc75,пальтишко демисезонное,30000
ab3e6941c11304c1519aef75,attipas,80000
252452a91c944a22c276d995,жилет теплый,50000


In [13]:
df_test_proc =preprocess_data(df_test)

In [14]:
df_test_proc.head()

,name
id,
285ea2e9935ccdeb8378c6a5,стремянка
adfb73820bbb831257df6e95,плита электрическая
783025601c36202f633fc6a5,диски r rays mazdaspeed touring х j
2f0cd2d2e15dc90afd847f95,batman archam knight
5c23a37902855a20172845a5,ледобур для зимней рыбалки


In [15]:
df_train_price = df_train.price

In [16]:
df_train_price.head()

id
3edeb34cf93f490ff760af85    199900
c98febd50dad3cc0ffc86085     35000
ade01e13912a46a99134cc75     30000
ab3e6941c11304c1519aef75     80000
252452a91c944a22c276d995     50000
Name: price, dtype: int64

In [17]:
df_all_names = df_train_proc.append(df_test_proc)['name']

In [18]:
df_all_names.head()

id
3edeb34cf93f490ff760af85                  сумка dg
c98febd50dad3cc0ffc86085                  комплект
ade01e13912a46a99134cc75    пальтишко демисезонное
ab3e6941c11304c1519aef75                   attipas
252452a91c944a22c276d995              жилет теплый
Name: name, dtype: object

In [19]:
cv = CountVectorizer()
cv.fit_transform(df_all_names)


<2498415x214093 sparse matrix of type '<class 'numpy.int64'>'
	with 6239701 stored elements in Compressed Sparse Row format>

In [20]:
cv.vocabulary_

{'легка': 125162,
 'негра': 141118,
 'ящиккасса': 214080,
 'дужкой': 98498,
 'technica': 54918,
 'коробкаот': 118190,
 'vitra': 59523,
 'пажарная': 150464,
 'warmluft': 60316,
 'stenn': 52996,
 'францияуже': 200248,
 'kingkong': 29633,
 'newline': 38936,
 'алголоджи': 66654,
 'prikinder': 44254,
 'полуболтнки': 159696,
 'курткаветровкашапка': 123244,
 'citysmart': 9963,
 'флешмодем': 199179,
 'спортивныйутепленныйв': 182813,
 'поролон': 160719,
 'датского': 91730,
 'клепатель': 112913,
 'пекинеса': 152564,
 'дубровка': 98451,
 'осьминог': 149002,
 'skx': 51052,
 'караблик': 109385,
 'автопрофи': 65041,
 'мылосамодельное': 138368,
 'кэт': 123845,
 'чазер': 204583,
 'riohyundai': 47135,
 'ременьgucciкожаный': 169889,
 'коняжки': 117591,
 'полукоммерческий': 159808,
 'ухаживающая': 197281,
 'этуаль': 212826,
 'sewing': 49938,
 'smal': 51320,
 'гпенза': 90010,
 'боюки': 76227,
 'букнтцветырозыоптомдоставка': 77680,
 'проекцией': 164005,
 'плеске': 156402,
 'igbgbgt': 26059,
 'tbrg': 54792,

In [21]:
train_data = cv.transform(df_train_proc.name)
test_data = cv.transform(df_test_proc.name)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_data, df_train_price, test_size=0.33, random_state=42)

LRclassifier=LogisticRegression(verbose=4)
LRclassifier.fit(X_train, y_train)

[LibLinear]

In [23]:
pickle.dump(LRclassifier, open('lrclassifier.pckl', 'wb'))

OverflowError: cannot serialize a bytes object larger than 4 GiB

In [24]:
rlmse_scorer(LRclassifier, X_test, y_test)

/home/katya/.pyenv/versions/katya_bank_project/lib/python3.5/site-packages/sklearn/metrics/regression.py:314: RuntimeWarning: divide by zero encountered in log
  return mean_squared_error(np.log(y_true + 1), np.log(y_pred + 1),


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [25]:
predict = LRclassifier.predict(cv.transform(df_test_proc.name))

In [29]:
result_predict = pd.DataFrame(data={'id': df_test.index, 'price': predict*0.01})
result_predict.to_csv('hackathon_predict_02_team3.csv', index=False)

In [28]:
result_predict.head()

,id,price
0,285ea2e9935ccdeb8378c6a5,500.0
1,adfb73820bbb831257df6e95,2000.0
2,783025601c36202f633fc6a5,4000.0
3,2f0cd2d2e15dc90afd847f95,1000.0
4,5c23a37902855a20172845a5,700.0
